In [4]:
# -*- coding: utf-8 -*-
# Importando as bibliotecas necessárias
import requests
from pypdf import PdfReader
from bs4 import BeautifulSoup
from language_tool_python import LanguageTool
import enchant
from urllib.parse import urljoin
import re
import os
from io import BytesIO
import csv


print("Importação de bibliotecas com sucesso")

#---------------------------------------------------------------------------------
# 1 DOWNLOAD DO ARQUIVO
#---------------------------------------------------------------------------------
#url = "https://csis-website-prod.s3.amazonaws.com/s3fs-public/2023-12/231206_Significant_Cyber_Events.pdf?VersionId=eHko0aa8DvqdG18bJbXUSZRmY2FNtNlL"
# 1.1 Buscando o link atual
url = "https://www.csis.org/programs/strategic-technologies-program/significant-cyber-incidents"
target_content = "Significant_Cyber_Events.pdf"
# 1.1.1 Obter o conteúdo HTML da página
response = requests.get(url)
html_content = response.text
# 1.1.2 Analisar o HTML usando BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
# 1.1.3 Encontrar o link que contém o texto desejado
target_link = None
for link in soup.find_all('a', href=True):
    if target_content in link['href']:
        target_link = link['href']
        break
# 1.1.4 Construir o URL completo, se um link for encontrado
if target_link:
    full_url = urljoin(url, target_link)
    print(f"1.1 Link encontrado: {full_url}")
else:
    print(f"1.1 Link com o conteúdo '{target_content}' não encontrado.")
    
# 1.2 Fazendo o download do arquivo PDF
response = requests.get(full_url)
pdf_file = BytesIO(response.content)

# 1.3 Salvando o arquivo PDF como csis.pdf
with open('csis.pdf', 'wb') as f:
    f.write(response.content)
print("1.2 download do pdf do CSIS realizado com sucesso")

#---------------------------------------------------------------------------------
# 2 TRANSFORMAÇÃO EM TEXTO
#---------------------------------------------------------------------------------
# 2.1 Transforma o .pdf em .txt
reader = PdfReader("csis.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"
with open('csis.txt', 'w', encoding='utf-8') as f:
    f.write(text)
print("2.1 Arquivo TXT gerado com sucesso")

# 2.2 Não há mais necessidade do arquivo pdf
os.remove("csis.pdf")
print("2.2 Arquivo csis.pdf foi eliminado")

#---------------------------------------------------------------------------------
# 3 EDIÇÃO DO ARQUIVO .TXT - Deixar apenas com o conteúdo bruto
#---------------------------------------------------------------------------------
# 3.1 ELIMINANDO O CABEÇALHO
# 3.1.1 Percebeu-se que o cabeçalho das versões sempre terminam com gictech@csis.org
file_path = 'csis.txt'
search_term = 'gictech@csis.org'
line_number = None
# 3.1.2 Buscamos a linha que está o referido termo
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for num, line in enumerate(lines, start=1):
        if search_term in line:
            line_number = num
            break
# 3.1.3 Se o termo foi encontrado, gravar no mesmo arquivo a partir da linha seguinte
if line_number is not None:
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines[line_number:])
    print(f"3.1 Cabeçalho Eliminado")
else:
    print(f"3.1 Cabeçalho não foi eliminado porque o termo gictech@csis.org não foi encontrado no arquivo.")

# 3.2 Eliminando o termo Center for Strategic and International Studies...
file_path = 'csis.txt'
search_phrase = 'Center for Strategic and International Studies (CSIS) | Washington, D.C.'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
# 3.2.1 Substituir a frase no conteúdo
new_content = content.replace(search_phrase, '')
# 3.2.2 Gravar as alterações no mesmo arquivo
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(new_content)
print(f'3.2 A frase Center for Strategic ... foi eliminada')

# 3.3 Eliminando espaços em branco
file_path = 'csis.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
# 3.3.1 Eliminar espaços em branco no início das linhas
lines = [line.lstrip() for line in lines]
# Gravar as alterações no mesmo arquivo
with open(file_path, 'w', encoding='utf-8') as file:
    file.writelines(lines)
print(f'3.3 Espaços em branco no início das linhas foram removidos')

#3.4 Junção de linhas
#3.4.1 Colocando os caracteres %$# no início de cada linha - também está sendo tratado alguns erros de inglês
with open('csis.txt', 'r', encoding='utf-8') as f:
    linhas2 = f.readlines()
for i in range(len(linhas2)):
    palavras = linhas2[i].split()
    if palavras[0] == 'Janu ary': linhas2[i] = '%$# January ' + ' '.join(palavras[1:])
    if palavras[0] == 'January': linhas2[i] = '%$# January ' + ' '.join(palavras[1:])
    if palavras[0] == 'February': linhas2[i] = '%$# February ' + ' '.join(palavras[1:])
    if palavras[0] == 'March 29': linhas2[i] = '%$# March 2022 ' + ' '.join(palavras[1:])
    if palavras[0] == 'March': linhas2[i] = '%$# March ' + ' '.join(palavras[1:])
    if palavras[0] == 'Mach': linhas2[i] = '%$# March ' + ' '.join(palavras[1:])
    if palavras[0] == 'April': linhas2[i] = '%$# April ' + ' '.join(palavras[1:])
    if palavras[0] == 'May': linhas2[i] = '%$# May ' + ' '.join(palavras[1:])
    if palavras[0] == 'June': linhas2[i] = '%$# June ' + ' '.join(palavras[1:])
    if palavras[0] == 'July': linhas2[i] = '%$# July ' + ' '.join(palavras[1:])
    if palavras[0] == 'August': linhas2[i] = '%$# August ' + ' '.join(palavras[1:])
    if palavras[0] == 'September': linhas2[i] = '%$# September ' + ' '.join(palavras[1:])
    if palavras[0] == 'October': linhas2[i] = '%$# October ' + ' '.join(palavras[1:])
    if palavras[0] == 'Octobe': linhas2[i] = '%$# October ' + ' '.join(palavras[1:])
    if palavras[0] == 'November': linhas2[i] = '%$# November ' + ' '.join(palavras[1:])
    if palavras[0] == 'December': linhas2[i] = '%$# December ' + ' '.join(palavras[1:])
    #algumas linhas não tratam o mês corretamente
    if palavras[0] == '2010': linhas2[i] = '%$# Year 2010 ' + ' '.join(palavras[1:])
    if palavras[0] == '2008.': linhas2[i] = '%$# Year 2008 ' + ' '.join(palavras[1:])
    if palavras[0] == '2007.': linhas2[i] = '%$# Year 2007 ' + ' '.join(palavras[1:])
    if palavras[0] == '2006.': linhas2[i] = '%$# Year 2006 ' + ' '.join(palavras[1:])
    if palavras[0] == '2005.': linhas2[i] = '%$# Year 2005 ' + ' '.join(palavras[1:])
    if palavras[0] == '2003': linhas2[i] = '%$# Year 2003 ' + ' '.join(palavras[1:])
    if palavras[0] == 'Summer': linhas2[i] = '%$# Summer ' + ' '.join(palavras[1:])
with open('csis.txt', 'w', encoding='utf-8') as f:
    for linha3 in linhas2:
        f.write(linha3)
#3.4.2 Juntando todas as linhas
with open('csis.txt', 'r', encoding='utf-8') as arquivo:
    linhas = arquivo.readlines()
linhas_sem_pulo = [linha.strip() for linha in linhas]
# 3.4.3 Substituindo o %$# por um enter \n
linhas_substituidas = [linha.replace('%$#', '\n') for linha in linhas_sem_pulo]
with open('csis.txt', 'w', encoding='utf-8') as arquivo_modificado:
    for linha in linhas_substituidas:
        arquivo_modificado.write(linha)
print("3.4 Linhas juntadas")

# 3.5 Eliminação dos problemas da segunda palavra
# 3.5.1 Erros de período. Alguns exemplos estão trazendo duração, vamos ficar somente com o início
delete_list=["-Apri", "– October", "and November"]
with open("csis.txt", "r", encoding='utf-8') as input_file:
    with open("csis2.txt", "w", encoding='utf-8') as output_file:
        for line in input_file:
            for word in delete_list:
                line = line.replace(word, "")
            output_file.write(line)
# 3.5.2 Segunda palavra tem que ter 4 caracteres
with open('csis2.txt', 'r', encoding='utf-8') as file_in:
    lines_out = []
    for line in file_in:
        words = line.split()
        if len(words) >= 2:
            # verificando o tamanho da segunda palavra
            if len(words[1]) == 4:
                # caso possua exatamente 4 caracteres, adiciona na lista sem modificações
                lines_out.append(line)
            elif len(words[1]) < 4 and len(words) >= 3:
                # caso possua menos de 4 caracteres e exista ao menos uma terceira palavra
                # junta a segunda e a terceira palavra e adiciona na lista
                new_line = f"{words[0]} {words[1] + words[2]} {' '.join(words[3:])}\n"
                lines_out.append(new_line)
            elif len(words[1]) > 4:
                # caso possua mais de 4 caracteres, remove o último caracter da segunda palavra
                new_line = f"{words[0]} {words[1][:-1]} {' '.join(words[2:])}\n"
                lines_out.append(new_line)
with open('csis.txt', 'w', encoding='utf-8') as file_out:
    file_out.writelines(lines_out)
os.remove('csis2.txt')
# 3.5.3 Erros persistentes
file_path = 'csis.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
for i in range(len(lines)):
    words = lines[i].split()
    if len(words) > 1:
        second_word = words[1]
        if "2023:" in second_word:
            words[1] = "2023"
        elif "29" in second_word:
            words[1] = "2022"
        elif "l2011." in second_word:
            words[1] = "2011"
        lines[i] = ' '.join(words) + '\n'
with open(file_path, 'w', encoding='utf-8') as file:
    file.writelines(lines)
print("3.5 Modificações na segunda palavra realizadas com sucesso")

#---------------------------------------------------------------------------------
# 4 RESOLVENDO PROBLEMAS DE ORTOGRAFIA - DEMORA UM POUCO - UTILIZAÇÃO DE DOIS CORRETORES
#---------------------------------------------------------------------------------
# 4.1 Leitura do conteúdo do arquivo
with open('csis.txt', 'r', encoding='utf-8') as file:
    texto = file.read()
# 4.1.1 Inicialização do LanguageTool no modo inglês dos EUA
tool = LanguageTool('en-US')
# 4.1.2 Verificação e correção de erros
correcoes = tool.check(texto)
texto_corrigido = tool.correct(texto)
# 4.1.3 Salvando o texto corrigido no arquivo
with open('ort.txt', 'w', encoding='utf-8') as file:
    file.write(texto_corrigido)
print("4.1 Correção ortográfica pela biblioteca language_tool_python terminada")
#os.remove('csis.txt')

# 4.2 resolvendo problemas de hífen - o segundo corretor estava transformando o hífen em "e"
with open('ort.txt', 'r', encoding='utf-8') as file:
    texto = file.read()
# 4.2.1 Substituir " -" por "-"
texto_corrigido = texto.replace(" -", "-")
# 4.2.2 Salvar o texto corrigido de volta no arquivo
with open('ort.txt', 'w', encoding='utf-8') as file:
    file.write(texto_corrigido)
print("4.2 Problema do hífen resolvido")

#---------------------------------------------------------------------------------
# 5 CRIANDO A BASE DE DADOS TOTAL DO CSIS EM CSV
#---------------------------------------------------------------------------------
input_file_path = 'ort.txt'
output_file_path = 'csisTotal.csv'
# 5.1 Inicializar o ID
id_counter = 1
# 5.2 Lista para armazenar os dados do CSV
csv_data = []
# 5.3 Processar cada linha do arquivo
with open(input_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Dividir a linha em palavras
        words = line.split()
        # Obter Ano, Mês e Descrição conforme as regras especificadas
        ano = words[1][:4] if len(words) > 1 else ''
        mes = words[0] if len(words) > 0 else ''
        descricao = ' '.join(words[2:])  # A descrição começa a partir da terceira palavra
        # Adicionar os dados à lista CSV
        csv_data.append([id_counter, ano, mes, descricao])
        # Incrementar o ID
        id_counter += 1
# 5.4 Escrever os dados no arquivo CSV
with open(output_file_path, 'w', encoding='utf-8', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['ID', 'Ano', 'Mês', 'Descrição'])
    csv_writer.writerows(csv_data)
print("5. Base de dados total do CSIS criada em csv com os campos ID, Ano, Mes e descrição")






Importação de bibliotecas com sucesso
1.1 Link encontrado: https://csis-website-prod.s3.amazonaws.com/s3fs-public/2023-12/231206_Significant_Cyber_Events.pdf?VersionId=eHko0aa8DvqdG18bJbXUSZRmY2FNtNlL
1.2 download do pdf do CSIS realizado com sucesso
2.1 Arquivo TXT gerado com sucesso
2.2 Arquivo csis.pdf foi eliminado
3.1 Cabeçalho Eliminado
3.2 A frase Center for Strategic ... foi eliminada
3.3 Espaços em branco no início das linhas foram removidos
3.4 Linhas juntadas
3.5 Modificações na segunda palavra realizadas com sucesso
4.1 Correção ortográfica pela biblioteca language_tool_python terminada
4.2 Problema do hífen resolvido
5. Base de dados total do CSIS criada em csv com os campos ID, Ano, Mes e descrição
